# SANA-FE Tutorial #
<a target="_blank" href="https://colab.research.google.com/github/SLAM-Lab/SANA-FE/blob/cpp/tutorial/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [ ]:
%pip install --extra-index-url https://test.pypi.org/simple/ sanafe==0.0.2
%pip install pyyaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/tutorial/arch.yaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/tutorial/snn.yaml
import sanafe
import yaml

import re  # For checking user exercises

### Running SANA-FE for the first time ###
Run SANA-FE for the first time using a minimal network and architecture we have provided. This will load the architecture, SNN and launch a short simulation. After the simulation has finished, a Dict summary of the simulated results is returned back and printed. As part of this tutorial, we will extend the SNN and architecture and will look at the hardware insights you can get from SANA-FE.

In [ ]:
# First, load an architecture description and build the chip (for simulation)
first_arch = sanafe.load_arch("arch.yaml")

# Second, load an SNN from file and upload it to the chip. We pass the
#  architecture as an extra argument so that SANA-FE can check the mappings are
#  to valid cores
hello_snn = sanafe.load_net("snn.yaml", first_arch)

# Third, create the spiking chip and upload the SNN
first_chip = sanafe.SpikingHardware(first_arch)
first_chip.load(hello_snn)

# Fourth and finally, run for 200 simulated timesteps
results = first_chip.sim(200)

In [ ]:
# Print the simulation results summary
print(f"Run results: {results}")

### Architecture Description File ###
In SANA-FE, you can describe different spiking architectures using a custom YAML-based architecture description format. The file `arch.yaml` contains a minimal example architecture based on the diagram below. Follow the three exercises at the top of the `arch.yaml` file to extend the architecture. These new elements are shown in dashed orange below.

![Example architecture](example_arch_small.png)

#### Exercises ####
1. Set the costs of a neuron update in the soma from 0.0 ns to 2.0 ns and from 0.0 pJ to 2.0 pJ. Note that ns=e-9 and pJ=e-12.
2. Duplicate tiles 2 times and each core 4 times within every tile (8 cores total). (For a hint look at files/loihi.yaml)
3. Define an additional synapse unit for compressed synapses. The energy and latency costs for reading a compressed synapse is 0.5 pJ and 2 ns. For a hint in how to define multiple hardware units, look at tutorial/loihi.yaml. This has three different synapse units defined.

After completing the exercises, run the next cell to see if all the checks pass!

In [ ]:
# Parse the extended architecture and manually check that each exercise was
#  completed correctly
green_text = "\033[92m"
red_text = "\033[31m"
default_text = "\033[0m"

with open("arch.yaml", "r") as arch_file:
    arch_details = yaml.safe_load(arch_file)

def check_exercise_1(arch_details):
    tiles = arch_details["architecture"]["tile"]
    cores = tiles[0]["core"]
    somas = cores[0]["soma"]
    soma_energy = somas[0]["attributes"]["energy_update_neuron"]
    soma_latency = somas[0]["attributes"]["latency_update_neuron"]
    if soma_energy == 2.0e-12 and soma_latency == 2.0e-9:
        print(f"{green_text}Exercise 1: PASS{default_text}")
    else:
        print(f"{red_text}Exercise 1: FAIL - Soma energy ({soma_energy}J) "
              f"and/or latency ({soma_latency}s) not set correctly{default_text}")

def parse_name_range(s):
    match = re.match(r"(\w+)\[(\d+)(?:\.\.(\d+))?\]", s)
    if match is None:
        return None, None
    else:
        return int(match.group(2)), int(match.group(3) or match.group(2))

def check_exercise_2(arch_details):
    tiles = arch_details["architecture"]["tile"]
    tile_name = tiles[0]["name"]
    range_start, range_end = parse_name_range(tile_name)
    passed = True
    total_tiles = 2
    if range_start is None:
        print(f"{red_text}Exercise 2: FAIL - Tile not duplicated{default_text}")
        return
    elif (range_end - range_start) + 1 != total_tiles:
        print(f"{red_text}Exercise 2: FAIL - Tile duplicated {1+(range_end-range_start)} "
              f"times, should be {total_tiles} times{default_text}")
        return

    cores = tiles[0]["core"]
    core_name = cores[0]["name"]
    range_start, range_end = parse_name_range(core_name)
    total_cores = 4
    if range_start is None:
        print(f"{red_text}Exercise 2: FAIL - Cores not duplicated{default_text}")
        passed = False
    elif (range_end - range_start) + 1 != total_cores:
        print(f"{red_text}Exercise 2: FAIL - Cores duplicated {1+(range_end-range_start)} "
              f"times, should be {total_cores} times{default_text}")
        passed = False

    if passed:
        print(f"{green_text}Exercise 2: PASS{default_text}")

def check_exercise_3(arch_details):
    tiles = arch_details["architecture"]["tile"]
    cores = tiles[0]["core"]
    synapses = cores[0]["synapse"]
    if len(synapses) != 2:
        print(f"{red_text}Exercise 3: FAIL - Expected to see 2 synapse units, "
              f"only found {len(synapses)}")
    else:
        # Get the new soma unit
        synapse = synapses[0]
        if synapse["name"] == "tutorial_synapse_uncompressed":
            synapse = synapses[1]
        synapse_energy = synapse["attributes"]["energy_process_spike"]
        synapse_latency = synapse["attributes"]["latency_process_spike"]

        if synapse_energy == 0.5e-12 and synapse_latency == 2.0e-9:
            print(f"{green_text}Exercise 3: PASS{default_text}")
        else:
            print(f"{red_text}Exercise 3: FAIL - New synapse energy ({synapse_energy}J) "
                  f"and/or latency ({synapse_latency}s) not set correctly{default_text}")

# Run checks
check_exercise_1(arch_details)
check_exercise_2(arch_details)
check_exercise_3(arch_details)


### SNN Description File ###
Next, we will finish the SNN defined in `snn.yaml` and map it to our updated architecture. The SNN is shown in the diagram below, with pre-defined elements drawn in black. Complete the four exercises listed in the file to add the orange elements shown and complete this small SNN.

![Example SNN](example_snn_small.png)


In [ ]:
green_text = "\033[92m"
red_text = "\033[31m"
default_text = "\033[0m"
with open("snn.yaml", "r") as snn_file:
    snn = yaml.safe_load(snn_file)

def check_exercise_1():
    net = snn["network"]
    group = net["groups"][1]
    neurons_found = len(group["neurons"])
    if len(group["neurons"]) != 2:
        print(f"{red_text}Exercise 1: FAIL - Should be 2 neurons in group 1, found {neurons_found}{default_text}")
        return

    # SANA-FE will check other aspects of the mapping, if it runs, it should be
    #  fine
    print(f"{green_text}Exercise 1: PASS{default_text}")

def check_exercise_2():
    net = snn["network"]
    edges = net["edges"]
    if len(edges) < 3:
        print(f"{red_text}Exercise 2: FAIL - Expected 3 edges but got {len(edges)}{default_text}")
        return

    # TODO: check weights are correct

    print(f"{green_text}Exercise 2: PASS{default_text}")

def check_exercise_3():
    net = snn["network"]
    group = net["groups"][0]
    neuron = group["neurons"][1]
    attributes = list(neuron.values())[0]
    if ("bias" not in attributes or attributes["bias"] != 0.5):
        print(f"{red_text}Exercise 3: FAIL - Neuron 0.1 bias not set to 0.5{default_text}")
    else:
        print(f"{green_text}Exercise 3: PASS{default_text}")

def check_exercise_4():
    net = snn["network"]
    group = net["groups"][1]
    from functools import reduce
    attributes = reduce(lambda a, b: {**a, **b}, group["attributes"])

    if attributes["synapse_hw_name"] == "tutorial_synapse_uncompressed":
        print(f"{red_text}Exercise 4: FAIL - Set group 1 synapse h/w to your "
              f"new synapse H/W unit{default_text}")
    else:
        print(f"{green_text}Exercise 4: PASS{default_text}")

check_exercise_1()
check_exercise_2()
check_exercise_3()
check_exercise_4()

### Python interface ###
SANA-FE v2 supports building SNNs in Python, in addition to file-based inputs. The example SNN has been recreated using the Python interface, optionally try completing the four exercises again but using the Python API.

As a reminder, these were:
#### Exercises ####
1. Define a new mapped neuron: 1.1. To do this, add another neuron to group 1 (i.e. increment the number of neurons in the group) and map neuron 1.1 to core 0.1
2. Add edges from neurons 0.0 and 0.1, both to neuron 1.1, with weights -2 & 3 respectively
3. Set the bias of neuron 0.1 to 0.5
4. Configure Group 1 to use the new compressed synapses that you defined H/W for in architecture description, instead of the uncompressed synapses used currently

In [ ]:
# Create an example SNN
in_attributes = {"threshold": 1.0, "reset": 0.0, "log_spikes": True, "log_potential": True}
out_attributes = {"threshold": 2.0, "reset": 0.0, "synapse_hw_name": "tutorial_synapse_uncompressed"}

# Create neuron groups
snn = sanafe.Network()
in_group = snn.create_neuron_group("in", 2, in_attributes)
out_group = snn.create_neuron_group("out", 1, out_attributes)

# Set neuron attributes
in_group.neurons[0].set_attributes(model_parameters={"bias": 0.2})

# Create connections
in_group.neurons[0].connect_to_neuron(out_group.neurons[0], {"weight": -1.0})

# Create mappings
arch = sanafe.load_arch("arch.yaml")
core = arch.tiles[0].cores[0]
in_group.neurons[0].map_to_core(core)
in_group.neurons[1].map_to_core(core)
out_group.neurons[0].map_to_core(core)

Once you've made all the changes, execute the code below to run your updated SNN on the updated arch.

In [ ]:
tutorial_chip = sanafe.SpikingHardware(arch)
tutorial_chip.load(snn)
results = tutorial_chip.sim(100, heartbeat=1)
print(results)

# Add code to check

### SNN Traces ###

TODO

Now that both an architecture and SNN have been defined, we can look at the
outputs SANA-FE can generate for these files. We use -o to specify an output
directory. First, we will just look at the run summary that SANA-FE writes
to a file, saving all output to the `tutorial` directory.

    python3 sim.py -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/run_summary.yaml

The file `run_summary.yaml` matches the printed summary at the end of the run.

Next, run another simulation but with the spike and potential trace flags set.
To enable spike traces, prepend (`-s`) to your run command. If the spike trace
flag is set, the simulator will record spikes for neurons with `log_spikes` set
to 1 in the SNN description. Similarly, the potential trace is set by
prepending (`-v`) to the run command. If enabled, SANA-FE logs the neuron
potentials of any voltage probes. Voltage probes are set by defining
`log_potential` to 1 for neurons in the SNN description.

In this example, two different flags are set simultaneously, i.e., (`-s -v`).
Therefore, two traces will be generated: `spikes.csv` and `potential.csv`.

    python3 sim.py -s -v -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/spikes.csv
    cat tutorial/potential.csv

Every line in the spike trace (`tutorial/spikes.csv`) has the format:

    <neuron>,<timestep spiked>

The potential trace (`tutorial/potential.csv`) has a column per probe and
one line per time-step i.e.:

    <neuron 0, timestep 0>,<neuron 1, timestep 0>
    <neuron 0, timestep 1>,<neuron 1, timestep 1>
    ...

Try visualizing `potential.csv` by plotting the dynamics of neurons 0.0 and
0.1 for 10 simulated timesteps, using your preferred plotting application.

### Hardware Traces ###
TODO

Next, run the same simulation but with message and performance traces
enabled instead. Message tracing is set by prepending `-m` to the command.
This will cause the simulator to record information about spike messages sent
by hardware. Performance tracing is set by prepending `-p` to the command.
Performance traces contain more detailed per-timestep information about
activity on the simulated design. Similar to the last example, we can combine
multiple traces, using `-m -p`.

    python3 sim.py -m -p -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/perf.csv
    cat tutorial/messages.csv

Two different traces will be generated: `perf.csv` and `messages.csv`.
Both files have similar formats, with one line per time-step and different
columns for various statistics. Try visualizing the `fired` and `total_energy`
fields over time.
